In [45]:
import numpy as np
import pandas as pd

In [46]:
class FormSquares:
    def __init__(self, df_result):
        self.df_result = df_result

    def run_square(self):
        lon_dict = {}
        lat_dict = {}

        x_max = np.max(self.df_result['line12'])
        y_max = np.max(self.df_result['line14'])

        print("x_max", x_max, "y_max", y_max)

        for i in range(len(self.df_result)):
            x = self.df_result['line12'][i]
            y = self.df_result['line14'][i]
            lon_dict[(x, y)] = self.df_result['lon'][i]
            lat_dict[(x, y)] = self.df_result['lat'][i]

        cor1_lon_series = []
        cor1_lat_series = []

        cor2_lon_series = []
        cor2_lat_series = []

        cor3_lon_series = []
        cor3_lat_series = []

        cor4_lon_series = []
        cor4_lat_series = []

        square_id_series = []
        x_series = []
        y_series = []
        count = 0

        for i in range(y_max):

            for j in range(x_max):
                square_id_series.append(str(j) + str(i))

                cor1_lon_series.append(lon_dict[j, i])
                cor1_lat_series.append(lat_dict[j, i])

                cor2_lon_series.append(lon_dict[j + 1, i])
                cor2_lat_series.append(lat_dict[j + 1, i])

                cor3_lon_series.append(lon_dict[j + 1, i + 1])
                cor3_lat_series.append(lat_dict[j + 1, i + 1])

                cor4_lon_series.append(lon_dict[j, i + 1])
                cor4_lat_series.append(lat_dict[j, i + 1])

                count += 1
                x_series.append(j)
                y_series.append(i)

        print("total square percentage ", count/len(self.df_result))

        data = {'square_id': square_id_series, 'x': x_series, 'y': y_series,
                'lon_cor1': cor1_lon_series, 'lat_cor1': cor1_lat_series,
                'lon_cor2': cor2_lon_series, 'lat_cor2': cor2_lat_series,
                'lon_cor3': cor3_lon_series, 'lat_cor3': cor3_lat_series,
                'lon_cor4': cor4_lon_series, 'lat_cor4': cor4_lat_series}

        df_square = pd.DataFrame(data=data)
        return x_max, y_max, df_square

In [47]:
class GenerateGrid:

    def __init__(self, grid_size):

        # initialize the grid size in meters.
        self.grid_size = grid_size

        self.lat_corner1 = 38.995418
        self.lon_corner1 = -77.041078

        self.lat_corner2 = 38.892845
        self.lon_corner2 = -76.909496

        self.lat_corner3 = 38.828800
        self.lon_corner3 = -76.9992657

        self.lat_corner4 = 38.934471
        self.lon_corner4 = -77.119045

    def compute_parameter(self, lon0, lat0, lon1, lat1):

        lon0 = np.radians(np.float64(lon0))
        lat0 = np.radians(np.float64(lat0))

        lon1 = np.radians(np.float64(lon1))
        lat1 = np.radians(np.float64(lat1))

        value = np.sin(lat0) * np.sin(lat1) + (np.cos(lat0) * np.cos(lat1) * np.cos(lon0 - lon1))

        dist = 6371.01 * 1000 * np.arccos(value)

        steps = int(dist/self.grid_size)
        t = self.grid_size/dist

        return t, steps

    def run_main(self):

        t12, s12 = self.compute_parameter(self.lon_corner1, self.lat_corner1, self.lon_corner2, self.lat_corner2)
        t14, s14 = self.compute_parameter(self.lon_corner1, self.lat_corner1, self.lon_corner4, self.lat_corner4)

        lon0 = self.lon_corner1
        lat0 = self.lat_corner1

        lon1 = self.lon_corner2
        lat1 = self.lat_corner2

        lon_t_series = []
        lat_t_series = []
        line12_id = []
        line14_id = []

        for i in range(s14 + 5):
            for j in range(s12):
                lat_t = (1 - (j * t12)) * lat0 + (j * t12) * lat1
                lon_t = (1 - (j * t12)) * lon0 + (j * t12) * lon1

                lat_t_series.append(lat_t)
                lon_t_series.append(lon_t)
                line12_id.append(j)
                line14_id.append(i)

            lon0 = (1 - (i * t14)) * self.lon_corner1 + (i * t14) * self.lon_corner4
            lat0 = (1 - (i * t14)) * self.lat_corner1 + (i * t14) * self.lat_corner4

            lon1 = (1 - (i * t14)) * self.lon_corner2 + (i * t14) * self.lon_corner3
            lat1 = (1 - (i * t14)) * self.lat_corner2 + (i * t14) * self.lat_corner3

        data = {'line12': line12_id, 'line14': line14_id, 'lon': lon_t_series, 'lat': lat_t_series}
        
        df_result = pd.DataFrame(data=data)
        x_max, y_max, df_square = FormSquares(df_result).run_square()

        df_square['x_max'] = x_max
        df_square['y_max'] = y_max

        return x_max, y_max, df_square

In [48]:
g = GenerateGrid(100)
x_max, y_max, df_square = g.run_main()

x_max 160 y_max 99
total square percentage  0.9838509316770186


In [49]:
df_square.to_csv("washington_grid_square_grid_side_size_100m.csv")